# Import

In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer


# Data exploring

In [ ]:
#data exploring
df = pd.read_csv('DATI.csv')

df

# Data cleaning

In [ ]:
# rimuoviamo le label in eccesso tenendo solo le prime 3 righe
def trova_terza(str):
    for _ in range(3):
        idx = str.rfind(',')
        str = str[:idx]
    str = str + ']'
    print(str)

    return str

In [ ]:
df['Label'] = df['Label'].apply(trova_terza)

# aggregamento dati

In [ ]:

def aggrega(df, colonna, batch_size=10):
    """
    Assegna la label più frequente ogni blocco di 10
    da specificare il df e la ['']
    """
    
    lista = []  
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]  
        
        labels = batch[colonna]  #label del batch
        counter = Counter(labels)     #trova la label più frequente 
        most_common_label = counter.most_common(1)[0][0]
        
        lista.extend([most_common_label] * len(batch))
    
    return pd.Series(lista, index=df.index)

In [ ]:
df['Label'] = aggrega(df, colonna='Label')

In [ ]:
df

# Data cleaning PT 2

In [ ]:
# la label è salvata come una stringa unica, quindi la puliamo e dividiamo con strip
# che dividerà in elementi di una lista
df['Label'] = df['Label'].apply(lambda x: x.replace('[',''))
df['Label'] = df['Label'].apply(lambda x: x.replace(']',''))
df['Label'] = df['Label'].apply(lambda x: x.replace('\'',''))

df

In [ ]:
df['Label'] = df['Label'].apply(lambda x: x.split(','))
df['Label']
#trasfroma in lista

In [ ]:
# come scelta progettuale ho eliminato le label non necessarie:

""" 
dato che facciamo una classificazione multi-classe dobbiamo dividere tutte le label e non ci serve sapere che dei
sensori sono in stato fermo, ma solo che il moviemento che ci interessa, quindi avanti/indietro/destra/ecc...
"""
def trasforma(lista):
    contatore = 0
    for elemento in lista:
        if elemento.strip() == 'fermo':
            lista.remove(elemento)
            contatore += 1
    
    if contatore == 3:
        elemento = 'fermo'
        return elemento
    
    return lista

In [ ]:
df['Label'] = df['Label'].apply(trasforma)
df

In [ ]:
df['Label'].value_counts()
#controllo se ci sono label vuote (no)

In [ ]:
# trasformazione label in binarie, per poi fornirle al modello
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(df['Label'])
print(binary_labels[0])

In [ ]:
#mostra le classi corrispondenti
print(mlb.classes_) 

# aggiuntiva per migliorare il training
non runnate a meno che sicuri

In [ ]:
df2 = df.drop('timestamp', axis = 1)
df2 = df2.drop('gyroX', axis = 1)
df2 = df2.drop('gyroY', axis = 1)
df2 = df2.drop('gyroZ', axis = 1)


In [ ]:
df2

In [ ]:
# Caricamento e suddivisione dei dati
x = df2.drop('Label', axis  = 1)
y = binary_labels

# train test split

In [ ]:
# Caricamento e suddivisione dei dati
# non avviare a meno che si voglia usare il modello fra virgolette
x = df.drop('Label', axis  = 1)
y = binary_labels

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

# normalizzazione

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
print(x_train.shape)
y_train.shape

# convertire i dati per il modello

In [ ]:
sequence_length = 10 #sequenza degli istanti temporali

def create_sequences(train, sequence_length):
    
    seq_temp = []
    for i in range(len(train) - sequence_length + 1):
        seq_temp.append(train[i:i + sequence_length])
    return np.array(seq_temp)

x_train = create_sequences(x_train, sequence_length)
x_test = create_sequences(x_test, sequence_length)

print(x_train.shape)
x_test.shape

In [ ]:
#scartiamo da y_test i dati in eccesso che non riescono a creare una sequenza intera
diff = len(y_test) - len(x_test)
print(diff)

if diff > 0:
    y_test = y_test[:-diff]

# addestramento

In [ ]:
'''# Creazione del modello LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 7), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(7, activation='sigmoid'))  


# Compilazione del modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Allenamento
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)
'''

# cosa dobbiamo fare
- implementare un early stopping al modello prima che cali d'accuracy
- giocare con l'ampiezza di una sequenza temporale (adesso è 10)
- aumentare le dimensioni del modello (mettiamo più strati, per farlo dopo uno strato bisogna mettere return_sequences)

In [ ]:
# Creazione del modello LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, 3), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(8, activation='sigmoid'))  


# Compilazione del modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Allenamento
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
# Valutazione

predizioni = model.predict(x_test)
soglia_corretto = 0.7
predicted_labels = (predizioni > soglia_corretto)

accuracy = accuracy_score(y_test, predicted_labels)

In [ ]:
print(f"Accuracy: {accuracy}")

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predicted_labels, target_names=mlb.classes_))
